In [1]:
import os

In [2]:
!cd /content
!rm -rf /content/mt

In [5]:
# clone if does not exist
if not os.path.isdir('/content/mt'):
    !git clone https://github.com/fbelderink/mt.git
%cd /content/mt
!git pull
!git checkout fb/dev
!git pull

/content/mt
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 21 (delta 11), reused 21 (delta 11), pack-reused 0
Unpacking objects: 100% (21/21), 2.07 KiB | 235.00 KiB/s, done.
From https://github.com/fbelderink/mt
   c5bd533..b46ba33  fb/dev     -> origin/fb/dev
Updating c5bd533..b46ba33
Fast-forward
 assignments/assignment3.py        |  2 +-
 assignments/assignment5.py        | 28 ----------------------------
 preprocessing/dataset/__init__.py |  1 +
 training/train.py                 | 27 ++++++++++++++++++---------
 4 files changed, 20 insertions(+), 38 deletions(-)
 delete mode 100644 assignments/assignment5.py
Already on 'fb/dev'
Your branch is up to date with 'origin/fb/dev'.
Already up to date.


In [6]:
import training.train as train
from utils.hyperparameters.ConfigLoader import ConfigLoader
from utils.hyperparameters.model_hyperparameters import FFModelHyperparameters
from utils.hyperparameters.train_hyperparameters import FFTrainHyperparameters
from google.colab import drive

In [14]:
drive_dir = '/content/drive'
drive.mount(drive_dir)

dataset_dir = os.path.join(drive_dir,'MyDrive/data/mt')

window_size = 3

model_params = FFModelHyperparameters(
    ConfigLoader("configs/ff/config.yaml").get_config())

train_params = FFTrainHyperparameters(
    ConfigLoader("configs/training/ff_train_config.yaml").get_config())

train.train(os.path.join(dataset_dir,f"train7k_w{window_size}.pt"),
            os.path.join(dataset_dir,f"val7k_w{window_size}.pt"),
            model_params,
            train_params)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
training on cuda:0
Number of Batches: 2094
Batch Size: 200
model name: None
total number of trainable parameters: 12099622
Layers:
source_embedding: Embedding(6797, 150)
target_embedding: Embedding(6461, 150)
fc_source: LinearLayer()
fc_source_bn: BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
dropout_source: Dropout(p=0.05, inplace=False)
fc_target: LinearLayer()
fc_target_bn: BatchNorm1d(600, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
dropout_target: Dropout(p=0.05, inplace=False)
fc1: LinearLayer()
fc1_bn: BatchNorm1d(1200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
dropout1: Dropout(p=0.05, inplace=False)
fc2: LinearLayer()
fc2_bn: BatchNorm1d(6461, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

Starting Training:

Epoch 1/100
steps: 0, epoch: 1
batch metrics: ac

KeyboardInterrupt: 

In [ ]:
from datetime import datetime
date = datetime.today().strftime('%d-%m-%Y')

In [ ]:
import torch
from assignments.assignment4 import *
from utils.file_manipulation import load_data
from preprocessing.dictionary import Dictionary

eval_all = True

data_de = load_data("data/raw/multi30k.dev.de")
data_en = load_data("data/raw/multi30k.dev.en")

dict_de = Dictionary.load(os.path.join(dataset_dir, "dicts/train_dict_de.pkl"))
dict_en = Dictionary.load(os.path.join(dataset_dir, "dicts/train_dict_en.pkl"))

if eval_all:
  determine_models_bleu(f"eval/checkpoints/{date}", data_de, data_en,
                      dict_de, dict_en, 3,
                      model_hyperparameters.window_size, True)
else:
  model = torch.load(f"eval/checkpoints/{date}/15_01_58.pth")
  test_model_bleu(model, data_de, data_en, dict_de, dict_en,
                 3, model_hyperparameters.window_size, True, None)


determine bleu of eval/checkpoints/13-06-2024/15_01_58.pth
Model BLEU: 0.2742871839806815
determine bleu of eval/checkpoints/13-06-2024/15_01_19.pth
Model BLEU: 0.2777224360095151
determine bleu of eval/checkpoints/13-06-2024/15_00_05.pth
Model BLEU: 0.2504418786978578
determine bleu of eval/checkpoints/13-06-2024/15_00_42.pth
Model BLEU: 0.2716663025217683
determine bleu of eval/checkpoints/13-06-2024/14_59_27.pth
Model BLEU: 0.21416791624619436


In [ ]:
!zip -r {dataset_dir}/checkpoints.zip eval/checkpoints/{date}/

  adding: eval/checkpoints/13-06-2024/ (stored 0%)
  adding: eval/checkpoints/13-06-2024/15_01_58.pth (deflated 7%)
  adding: eval/checkpoints/13-06-2024/15_01_19.pth (deflated 7%)
  adding: eval/checkpoints/13-06-2024/15_00_05.pth (deflated 7%)
  adding: eval/checkpoints/13-06-2024/15_00_42.pth (deflated 7%)
  adding: eval/checkpoints/13-06-2024/14_59_27.pth (deflated 8%)


In [ ]:
!rm -rf eval/checkpoints/{date}